In [1]:
from trainer.data import load_data, load_data_from_list
from trainer.policy import get_model
from trainer.env import make_env
from trainer.preprocessing import make_observation_window
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from trade_tester.tester_base_class import render_candles
import xarray as xr

from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras import layers
import tensorflow as tf
from keras.optimizers import Adam
# отключим GPU (нужно для платформы Apple M1)
# tf.config.set_visible_devices([], 'GPU')

ModuleNotFoundError: No module named 'stable_baselines3'

In [2]:
load_data_kwargs = dict(
    path = 'klines/',
    symbol = 'DOGEUSDT',
    tf='15m',
    preprocessing_kwargs = dict(
        bb = dict(period=20, render=True, deviation=1.8),
        rsi = dict(period=14, render=True, separately=True, color='blue'),
        # ma = dict(period=20, render=True, color='red'),
        obv = dict(render=True, color='green', separately=True),
    ),
    split_validate_percent = 0,
)
train_klines, val_klines, indicators, dataset = load_data(**load_data_kwargs)

In [ ]:
train_klines

In [5]:
env_kwargs = dict(
    env_class='TradingEnv2BoxAction',
    tester='BBTester',
    klines=train_klines,
    data=dataset,
    indicators=indicators,
    b_size=1000,
)
env = make_env(**env_kwargs)
# model = get_model(env=env)
model = PPO('MlpPolicy', env)

In [ ]:
done=False
obs = env.reset()
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step([action])
    env.render()

In [6]:
model.policy._modules

OrderedDict([('features_extractor',
              FlattenExtractor(
                (flatten): Flatten(start_dim=1, end_dim=-1)
              )),
             ('pi_features_extractor',
              FlattenExtractor(
                (flatten): Flatten(start_dim=1, end_dim=-1)
              )),
             ('vf_features_extractor',
              FlattenExtractor(
                (flatten): Flatten(start_dim=1, end_dim=-1)
              )),
             ('mlp_extractor',
              MlpExtractor(
                (shared_net): Sequential()
                (policy_net): Sequential(
                  (0): Linear(in_features=1, out_features=64, bias=True)
                  (1): Tanh()
                  (2): Linear(in_features=64, out_features=64, bias=True)
                  (3): Tanh()
                )
                (value_net): Sequential(
                  (0): Linear(in_features=1, out_features=64, bias=True)
                  (1): Tanh()
                  (2): Linear(in_features=6

In [ ]:
# Извлеките параметры политики
policy_parameters = model.policy.state_dict()
# Выведите веса каждого слоя
for name, weights in policy_parameters.items():
    print(f"{name}: {weights}")

In [ ]:
# Исследование структуры model.policy
def explore_structure(obj, indent=0):
    for name in dir(obj):
        if not name.startswith("__"):
            value = getattr(obj, name)
            if not callable(value):
                print("  " * indent, f"{name}: {type(value)}")
                if isinstance(value, torch.nn.Module):
                    explore_structure(value, indent + 1)

# print("Policy structure:")
# explore_structure(model.policy)

In [ ]:
obs = env.reset()
obs = th.tensor(obs)
obs = obs.transpose(1, 2)[0]
for channel in range(obs.shape[0]):
    print()

In [ ]:
from trainer.nn import CustomNN

nn = CustomNN(np.array([[9, 300], [9, 300]]))

In [ ]:
nn.named_parameters

In [ ]:
for name, param in nn.named_parameters():
    num_params = param.numel()
    print(num_params)

In [ ]:
kwargs = dict(
    symbols = ['DOGEUSDT', 'DOGEBTC', 'BTCUSDT'],
    tfs = ['15m', '30m', '1h', '4h'],
    preprocessing_kwargs = dict(
                    bb = dict(period=20, deviation=2),
                    rsi = dict(period=14),
                    ma = dict(period=20),
                    obv = dict(),
                ),
)
dfs = load_data_from_list(**kwargs)

In [ ]:
# find min and max date over dfs
min_dates = []
for df in dfs:
    min_dates.append(df['date'].min())
min_date = max(min_dates)

# shrink dataset so that the minimum date is the same
dfs2 = []
for df in dfs:
    dfs2.append(df[df['date'] >= min_date].reset_index(drop=True))
dfs = dfs2

In [ ]:
len(dfs[0])

In [ ]:
dataset = []
window = 100
length = len(dfs[0]) - window
for i in range(1600+window+1, window+length):
    a = make_observation_window(dfs, date=dfs[0].iloc[i][0], window=window)
    dataset.append(a)

In [ ]:
dataset = np.array(dataset)
dataset.shape

In [ ]:
# Создаем xarray.DataArray с метками
date = dfs[0][:dataset.shape[0]]['date'].values
dataset = xr.DataArray(
    dataset,
    coords={'date': date},
    dims=['date', 'n', 'chanel']
)
dataset.to_netcdf('dataset.nc')

In [ ]:
dataset = xr.open_dataarray('dataset.nc')

In [ ]:
dataset = dataset[:1000]

In [ ]:
# Разделение данных на обучающую и тестовую выборки
split_index = int(0.8 * dataset.shape[0])
train_data = dataset[:split_index].to_numpy()
test_data = dataset[split_index:].to_numpy()

In [ ]:
train_data.shape

In [ ]:
def create_conv1d_autoencoder(input_shape=(100, 144)):
    # Encoder
    encoder = Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPooling1D(pool_size=2, padding='same'),
        layers.Dropout(0.1),
        layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')
    ])
    
    # Decoder
    decoder = Sequential([
        layers.Conv1D(32, kernel_size=3, activation='relu', padding='same'),
        layers.UpSampling1D(size=2),
        layers.Dropout(0.1),
        layers.Conv1D(144, kernel_size=3, activation='sigmoid', padding='same')
    ])
    
    # Autoencoder
    autoencoder = Sequential([
        encoder,
        decoder
    ])
    
    return autoencoder

In [ ]:
df = pd.DataFrame(decoded[0, :, :4], columns=['open', 'high', 'low', 'close']).reset_index()
df = df.rename(columns={'index': 'date'})

df2 = pd.DataFrame(test_data[0, :, :4], columns=['open', 'high', 'low', 'close']).reset_index()
df2 = df2.rename(columns={'index': 'date'})

In [ ]:
render_candles(df)

In [ ]:
render_candles(df2)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(decoded[0].transpose(), cmap='gray')

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(test_data[0].transpose(), cmap='gray')